In [2]:
!apt-get update
!apt install chromium-chromedriver
!pip install selenium

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:11 http://security.ubuntu.com/ubuntu bionic-security/restricted amd64 Packages [665 kB]
Hit:12 http://ppa.launchpad.net

In [3]:
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
import pandas as pd
import time
import os
import numpy as np

#exception 처리 대비
from selenium.common.exceptions import UnexpectedAlertPresentException, NoSuchElementException

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
kospi = pd.read_csv('/content/drive/Shareddrives/🥑PSAT_DM_2728🥑/Dataset/코스피_코스닥_코드/kospi.csv', encoding = 'utf-8')
kospi2 = pd.read_csv('/content/drive/Shareddrives/🥑PSAT_DM_2728🥑/Dataset/코스피_코스닥_코드/코스피.csv', encoding = 'cp949')

In [6]:
kospi2 = kospi2.drop(['표준코드', '단축코드', '한글 종목명', '영문 종목명', '상장일', '시장구분', '증권구분', '소속부', '주식종류', '액면가', '상장주식수'], axis = 1)

In [7]:
kospi_final = pd.merge(kospi2,kospi, how='left', left_on='한글 종목약명', right_on='Name')
kospi_final.dropna(subset = ["extract"], inplace=True)
kospi_final = kospi_final.reset_index(drop = True, inplace = False)

In [21]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import urllib.request


chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless') #내부 창을 띄울 수 없으므로 설정
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
# 아래 셀에서 Mozilla ~ 로 시작하는 부분을 자신의 컴퓨터에 맞게 바꿔주어야 함.
# http://www.useragentstring.com/
# 위 링크 들어가서 확인하기
chrome_options.add_argument(f'user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.81 Safari/537.36')
driver = webdriver.Chrome('chromedriver',chrome_options=chrome_options)
driver.implicitly_wait(10)

title_news = []
date_news = []
content_news = []
company = []

url_base1 = 'https://finance.naver.com/item/news_news.naver?code='
url_base2 = '&page='
url_base3 = '&sm=title_entity_id.basic&clusterId='

# 여기서 자신이 맡은 숫자로 바꿔주기
## EX) 500부터 1000까지라면 for k in range(500, 1000, 1)
###########################################
# 숫자 그대로 넣으세요!! 다 고려해서 배정한 거임!!! #
#          괜히 1 더해서 넣지 마 !!!!!!      #
#########################################
for k in range(368, 369, 1):
  print('-'*30)
  print(f'{k}번째 입니다.')
  print(f"\n {kospi_final['Name'][k]}")
  for i in range(1, 200, 1):
    url = url_base1 + kospi_final['extract'][k] + url_base2 + str(i) + url_base3
    driver.get(url)
    time.sleep(2)
    try:
      num = 0
      dates = driver.find_element_by_xpath(f'/html/body/div/table[1]/tbody/tr[1]/td[3]').text
    except:
      num = 99
    
    if num == 0:
      dates = driver.find_element_by_xpath(f'/html/body/div/table[1]/tbody/tr[1]/td[3]').text
      date = pd.to_datetime(dates)
      if (date >= pd.to_datetime('2021-10-01')):
        for j in range(1, 21, 1):
          try:
            url = url_base1 + kospi_final['extract'][k] + url_base2 + str(i) + url_base3
            driver.get(url)
            num = 1
            elem = driver.find_element_by_xpath(f'/html/body/div/table[1]/tbody/tr[{j}]/td[1]/a')
          except:
            url = url_base1 + kospi_final['extract'][k] + url_base2 + str(i) + url_base3
            driver.get(url)
            try:
              num = 2
              elem = driver.find_element_by_xpath(f'/html/body/div/table[1]/tbody/tr[{j}]/td/table/tbody/tr[1]/td[1]/a')
            except:
              num = 3
          if num == 1:
            companies = kospi_final['Name'][k]
            company.append(companies)
            driver.find_element_by_xpath(f'/html/body/div/table[1]/tbody/tr[{j}]/td[1]/a').click()
            time.sleep(2)
            titles = driver.find_element_by_xpath(f'//*[@id="content"]/div[2]/table/tbody/tr[1]/th/strong').text
            print('\n제목:')
            print(titles)
            title_news.append(titles)
            dates = driver.find_element_by_xpath(f'//*[@id="content"]/div[2]/table/tbody/tr[2]/th/span/span').text
            date = pd.to_datetime(dates)
            print('\n날짜')
            print(date)
            date_news.append(date)
            contents = driver.find_element_by_id('news_read').text
            print('\n본문')
            print(contents)
            content_news.append(contents)
          elif num == 2:
            companies = kospi_final['Name'][k]
            company.append(companies)
            titles = driver.find_element_by_xpath(f'/html/body/div/table[1]/tbody/tr[{j}]/td/table/tbody/tr[1]/td[1]/a').text
            print('\n제목:')
            print(titles)
            title_news.append(titles)
            driver.find_element_by_xpath(f'/html/body/div/table[1]/tbody/tr[{j}]/td/table/tbody/tr[1]/td[1]/a').click()
            time.sleep(2)
            dates = driver.find_element_by_xpath(f'//*[@id="content"]/div[2]/table/tbody/tr[2]/th/span/span').text
            date = pd.to_datetime(dates)
            print('\n날짜')
            print(date)
            date_news.append(date)
            contents = driver.find_element_by_id('news_read').text
            print('\n본문')
            print(contents)
            content_news.append(contents)
          elif num == 3:
            break
      else:
        break
    elif num == 99:
      break
            



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: DeprecationWarning: use options instead of chrome_options
  from ipykernel import kernelapp as app


------------------------------
368번째 입니다.

 삼성전자


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:43: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:48: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:56: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:68: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:70: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:74: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
/usr/local/lib/python3.7/dis


제목:
"삼성전자 잡자"­…日 소니, 대만 TSMC와 파운드리 협공

날짜
2021-11-10 23:02:00

본문
일본의 대표 전자회사 소니가 세계 최대 파운드리(반도체 위탁생산) 기업인 대만 TSMC와 합작회사를 세우고 삼성전자에 협공을 가한다. 글로벌 반도체 대란 와중에 일본이 TSMC의 생산 시설을 유치하면서 반도체 공급망을 자국에 유리한 방향으로 가져오려는 시도로 풀이된다.

10일 업계와 현지 언론에 따르면 TSMC와 소니는 구마모토현에 총 70억달러(한화 약 8조2600억원) 규모로 22~28나노 반도체 공장을 건설한다. 이곳에서 생산될 반도체는 첨단 제품은 아니지만 자동차와 정보기술(IT) 제품용으로 산업계에서 널리 사용된다. 내년 착공해 2024년 말부터는 양산을 시작할 예정이다.

두 회사는 해당 공장을 운영하는 합작회사를 구마모토현에 설립하며 소니가 5억달러(약 5900억원)를 출자한다. 소니의 반도체 자회사 소니세미컨덕터솔루션스(SSS)가 지분 20% 미만을 취득할 예정이며 주식의 과반은 TSMC가 보유해 경영권을 갖는다.

그간 일본 정부는 안정적 반도체 수급을 목표로 TSMC의 자국 유치에 공 들여왔다. 반도체 수급난이 발생하면 일본에 우선 공급하는 조건으로 공장 정비비의 절반 정도를 지원하기로 했다.

내친 김에 일본 정부는 다음달 임시국회에서 관련 개정 법안도 제출할 예정이다. 보조금 규모가 정식 결정되면 TSMC는 일본 정부로부터 최대 수천억엔의 보조금을 받게 된다. 일본이 해외 기업에 실시하는 지원으로는 사상 최대 규모다.

TSMC는 현재 90% 이상의 반도체를 대만에서 생산하고 있다. 해외에선 중국 장쑤성 난징시와 미국 애리조나주에 새로운 공장을 짓고 있다. 일본의 공장이 완공되면 해외 주력 생산거점으로는 미·중에 이어 세 번째가 된다.

소니, TSMC 로고 [사진=로이터 연합뉴스]

니혼게이자이는 이번 결정에 대해 "반도체 부족이 세계적 문제가 되는 가운데 경제안보 관점에서 공장을 유치한 것"이라며 "각국이 TSMC 

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:62: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:86: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:90: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead



제목:
이재용 약속 지켰다…삼성, 스타트업 400곳 육성


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:92: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:97: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead


스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.

제목:
공매도 펀치에 바닥까지 추락한 삼성전자…더 떨어질까

날짜
2021-10-06 23:11:00

본문
삼성전자, 공매도 잔고 연중 저점 대비 3배 넘게 늘어
대차거래 상위 10종목 중 잔고금액 가장 높아
지난 5일 기준 외국인 대차거래 비중 70% 수준[이데일리 유준하 기자] 국민 대장주 삼성전자(005930)의 9월 말 공매도 잔고 금액이 연중 저점을 찍은 지난 8월 말에 비해 3배 넘게 증가한 것으로 나타났다. 특히나 이날 국감에서는 외국인이 삼성전자 주식 1조7611억원 어치를 공매도한 것으로 나타나 눈길을 끌었다.

나아가 외국인 비중이 높은 대차거래에서도 상위 10종목 중 삼성전자가 가장 높은 잔고를 기록하는 등 향후에도 추가 공매도가 나올 가능성이 높아 주의해야 한다는 조언이 나온다.

[그래픽=이데일리 김정훈 기자]
6일 마켓포인트에 따르면 삼성전자 주가는 전거래일 대비 1.25%(900원) 내린 7만1300원에 거래를 마쳤다. 7만1000원대는 지난해 12월4일 종가(7만1500원)와 유사한 수치로 연중 최저가를 경신한 것이다. 지난달 30일 보합 마감을 제외하면 5거래일 연속 내림세를 이어갔다.

수급별로는 외국인의 매도세가 거셌다. 외국인은 이날 733억원 어치를 순매도했다. 기관은 251억원 순매도를 했으며 개인만 나홀로 997억원 어치를 사들였다. 외국인은 이날을 포함해 6거래일 연속 순매도를 이어갔다.

이처럼 외국인이 연일 순매도세를 보이는 가운데 대차거래 비중 역시 외국인이 높은 것으로 나타났다. 금융투자협회에 따르면 지난 5일 기준으로 최근 한달 간 외국인 대차거래 비중은 약 65%~71% 수준이다.

대차거래란 기관이 주식을 빌려 재상환하는 계약을 의미한다. 금융투자협회 관계자는 “개인이 주식을 빌리는 행위를 대주라고 한다면 대차 거래는 기관이 주식을 빌리는 계약을 의미한다”고 설명했다. 대차잔고가 늘어난다고 해서 무조건 공매도가 늘어나진 않지만 외국계 혹은

In [22]:
df = {'company': company, 'title': title_news, 'content': content_news, 'date' : date_news}
df = pd.DataFrame(df)

In [23]:
df = df.reset_index(drop = True, inplace = False)

In [24]:
df

,company,title,content,date
0,삼성전자,"""삼성전자 잡자""­…日 소니, 대만 TSMC와 파운드리 협공",일본의 대표 전자회사 소니가 세계 최대 파운드리(반도체 위탁생산) 기업인 대만 TS...,2021-11-10 23:02:00
1,삼성전자,"삼성이 키운 21개 스타트업, 성장 스토리 공유",'C랩 스타트업 데모데이' 열려\n10일 서울 서초구 '삼성전자 서울R&D캠퍼스'에...,2021-11-10 17:59:00
2,삼성전자,"이재용 약속 지켰다…삼성, 스타트업 400곳 육성",C랩 스타트업 데모데이 개최\n\n생태계 활성화로 고용 촉진\n\n국내 스타트업(신...,2021-11-10 17:25:00
3,삼성전자,아이폰 닦는 천 2만원에 파는 애플…삼성 “이런걸 돈 받나”,[갤럭시클럽 캡처]\n\n\n[헤럴드경제=박혜림 기자] “애플은 2만원에 파는 ‘광...,2021-11-10 17:52:00
4,삼성전자,"김현석 삼성전자 사장 ""마이크로LED 시장 확대할 것""","닛케이 세계경영자회의 강연\n""제품들 연결해 서비스 확대""\n\n김현석 삼성전자 소...",2021-11-10 17:35:00
...,...,...,...,...
1873,삼성전자,"“이 정도일 줄은…” 삼성 폴더블폰, 나오자마자 100만대 판매",‘갤럭시Z플립3’. [삼성전자 제공]\n\n\n[헤럴드경제=박혜림 기자] 삼성전자의...,2021-10-01 09:59:00
1874,삼성전자,"""삼성전자, 내년 역대 최대 실적 확실시""","[헤럴드경제=이현정 기자]한화투자증권은 1일 삼성전자에 대해 내년 메모리, 시스템 ...",2021-10-01 09:52:00
1875,삼성전자,"삼성전자, 반도체 수출 호조에도 7.3만원대 '털썩'",[특징주][이데일리 이은정 기자] 삼성전자(005930)가 장중 7만4000원을 하...,2021-10-01 09:43:00
1876,삼성전자,"[K-수소동맹]⑫뜨뜻미지근 삼성, 냉랭한 LG…왜?","삼성물산, 대표자 없이 단일기업 자격 참여\nLG는 아예 불참…'SK 불편한 관계 ...",2021-10-01 07:45:00


In [25]:
# 파일명만 코스닥(자신이 맡은 범위)로 써주세요
## EX) 코스피(500~1000).csv

df.to_csv('/content/drive/Shareddrives/🥑PSAT_DM_2728🥑/Dataset/[종목크롤링]/코스피/코스피(368~368).csv', index = False, encoding = 'utf-8')